# Día 7: Fórmulas dinámicas y hojas interactivas en Excel

### Objetivos:
Automatizar reportes de excel que incluyan

*   Múltiples hojas (datos, resumen e indicadores)
*   Formulas como **suma, sumar.si, buscarv**
*   KPIs calculados y posicionados
*   Celdas automáticas con formato y lógica dinámica



In [33]:
!pip install xlsxwriter

#### Paso 1: simulamos un dataset

In [34]:
import pandas as pd
import numpy as np

In [35]:
# Creando el dataset

np.random.seed(42)
productos = ['Laptop', 'Tablet', 'Monitor', 'Teclado', 'Mouse']
df = pd.DataFrame({
    'Producto': np.random.choice(productos, 100),
    'Ventas': np.random.randint(1000, 10000, 100)
})

In [36]:
df.tail()

,Producto,Ventas
95,Monitor,8125
96,Mouse,2930
97,Tablet,7287
98,Tablet,9800
99,Laptop,5282


#### Paso 2: Exportar a Excel con hoja de resumen + fórmula dinámica

In [45]:
with pd.ExcelWriter('reporte_con_formulas.xlsx', engine='xlsxwriter') as writer:
    # Hoja 1: datos
    df.to_excel(writer, sheet_name='Datos', index=False)
    workbook = writer.book
    hoja_datos = writer.sheets['Datos']

    # Hoja 2: resumen
    hoja_resumen = workbook.add_worksheet('Resumen')

    # Encabezados
    hoja_resumen.write('A1', 'Producto')
    hoja_resumen.write('B1', 'Total Ventas')

    # Insertar productos únicos y fórmulas SUMAR.SI
    for i, producto in enumerate(productos):
        hoja_resumen.write(i+1, 0, producto)
        hoja_resumen.write_formula(i+1, 1,
            f'=SUMAR.SI(Datos!A:A,Resumen!A{i+2},Datos!B:B)'
        )

    # Formato moneda
    formato_moneda = workbook.add_format({'num_format': '$#,##0'})
    hoja_resumen.set_column('B:B', 15, formato_moneda)



### Ejericio 2: Insertar un KPI destacado (total general)


In [46]:
# KPI en celda separada (E1)
hoja_resumen.write('D1', 'Total General')
hoja_resumen.write_formula('E1', f'=SUM(B2:B{len(productos)+1})', formato_moneda)

# Dar estilo a KPI
formato_kpi = workbook.add_format({
    'bold': True, 'font_color': 'white', 'bg_color': '#4472C4',
    'align': 'center', 'valign': 'vcenter'
})
hoja_resumen.set_column('D:E', 20)
hoja_resumen.write('D1', 'Total General', formato_kpi)
hoja_resumen.write_formula('E1', f'=SUM(B2:B{len(productos)+1})', formato_moneda)


0

### Ejercicio 3: Agregar una funcion BUSCARV

Agrega una tabla donde el usuario escriba un producto y vea su total de ventas automáticamente

In [47]:

hoja_resumen.write('G1', 'Buscar Producto:')
hoja_resumen.write('G2', '')  # Celda editable
hoja_resumen.write('H1', 'Ventas Totales:')
hoja_resumen.write_formula('H2', '=BUSCARV(Datos!A:A,G2,Datos!B:B)')


0